### Run this notebook to test the docker image.

In [2]:
import urllib.request, json, requests

# get the available questions and answers for a given topic using SQUAD v2.0
URL = 'https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json'
with urllib.request.urlopen(URL) as url:
    data = json.load(url)

def get_qa(topic, data):
    q = []
    a = []
    for d in data['data']:
        if d['title']==topic:
            for paragraph in d['paragraphs']:
                for qa in paragraph['qas']:
                    if not qa['is_impossible']:
                        q.append(qa['question'])
                        a.append(qa['answers'][0]['text'])
            return q,a

ls_questions, ls_answers = get_qa(topic='Premier_League', data=data)

In [3]:
# what are the sample questions?
ls_questions[:5]

['How many club members are there?',
 'How many matches does each team play?',
 'What days are most games played?',
 'Who sponsors the Premier League?',
 'What name is the Premier League known as outside of the United Kingdom?']

In [4]:
# what are the sample answers?
ls_answers[:5]

['The Premier League is a corporation in which the 20 member clubs act as shareholders',
 'Teams play 38 matches each (playing each team in the league twice, home and away), totalling 380 matches in the season.',
 'Most games are played on Saturday and Sunday afternoons; others during weekday evenings',
 'It is currently sponsored by Barclays Bank and thus officially known as the Barclays Premier League and is colloquially known as the Premiership.',
 'Outside the UK it is commonly referred to as the English Premier League (EPL).']

Ensure that the docker image is up and running before running the cells below!

In [8]:
# To set the context of the Q&A app, we "define the context" by using the \
# Stanford Question Answering Dataset v2.0

json_data = {
  'questions':ls_questions,
  'answers':ls_answers,
}

response = requests.post(
  'http://localhost:8000/set_context',
  json=json_data
)

response.json()

{'message': 'Search context set'}

In [9]:
# Post a new question.
# This will retrieve the "most similar question" and its "corresponding answer" \
# to the given custom question.

new_post_questions = [
    "what can i see here?",
    "how many teams are playing?",
]

json_data = {
  'questions':new_post_questions,
}

response = requests.post(
  'http://localhost:8000/get_answer',
  json=json_data
)

for d in response.json():
  print('\n'.join(["{} : {}".format(k, v) for k,v in d.items()])+'\n')

orig_q : what can i see here?
best_q : Where can the third lion be seen?
best_a : the third is symbolised by the captain of the title winning team as he raises the trophy, and its gold crown, above his head at the end of the season

orig_q : how many teams are playing?
best_q : How many matches does each team play?
best_a : Teams play 38 matches each (playing each team in the league twice, home and away), totalling 380 matches in the season.

